In [1]:
!pip install pyngrok


In [2]:
# Import necessary libraries
from flask import Flask, request, render_template
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
import joblib
import os
import numpy as np
import pandas as pd
from pyngrok import ngrok
import socket

# **Model Train,Test and Save**

In [3]:
# Step 1: Load the dataset
df = pd.read_csv('/kaggle/input/terativeimputer-credit1/IterativeImputer_credit(1).csv')

# Step 2: Ensure only feature columns are used (excluding the target column 'loan_status')
X = df.drop(columns=['loan_status'])  # Use only features
y = df['loan_status']

# Step 3: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

# Step 4: Apply MinMaxScaler for scaling
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Perform PCA for dimensionality reduction
pca = PCA(n_components=25)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Step 6: Train an XGBoostClassifier
xgb_classifier = XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=3, subsample=0.8, colsample_bytree=0.8, gamma=0, reg_lambda=1, objective='binary:logistic', random_state=42)
xgb_classifier.fit(X_train_pca, y_train)

# Step 7: Evaluate the model
y_pred_train = xgb_classifier.predict(X_train_pca)
y_pred_test = xgb_classifier.predict(X_test_pca)

train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

train_f1_score = f1_score(y_train, y_pred_train, average='weighted')
test_f1_score = f1_score(y_test, y_pred_test, average='weighted')

# Print the results
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Training F1 Score: {train_f1_score}")
print(f"Test F1 Score: {test_f1_score}")
print("Classification Report (Test Set):")
print(classification_report(y_test, y_pred_test))

# Step 8: Save the model and transformers to files using joblib
joblib.dump(xgb_classifier, 'loan_model.pkl')
joblib.dump(pca, 'pca_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

Training Accuracy: 0.9526179051681095
Test Accuracy: 0.9516730512098662
Training F1 Score: 0.9362463981557109
Test F1 Score: 0.9352080137645273
Classification Report (Test Set):


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.98      0.98      0.98      9039
         1.0       0.95      0.99      0.97    120219
         2.0       0.00      0.00      0.00       243
         3.0       0.58      0.22      0.32       147
         4.0       0.67      0.22      0.33       383
         5.0       0.95      0.99      0.97     41504
         6.0       0.00      0.00      0.00      1250
         7.0       0.70      0.03      0.06      1679
         8.0       0.00      0.00      0.00       471
         9.0       0.00      0.00      0.00      2316

    accuracy                           0.95    177251
   macro avg       0.48      0.34      0.36    177251
weighted avg       0.93      0.95      0.94    177251



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['scaler.pkl']

In [4]:
# Select the first row from the test data
first_test_row = X_test.iloc[4, :].values.reshape(1, -1)
true_label = y_test.iloc[0]

# Preprocess the first row (scaling and PCA transformation)
first_test_row_scaled = scaler.transform(first_test_row)
first_test_row_pca = pca.transform(first_test_row_scaled)

# Make a prediction using the trained model
predicted_label = xgb_classifier.predict(first_test_row_pca)

# Display the features of the first test row, the true label, and the predicted label
print("First Test Row (original features):")
print(X_test.iloc[4, :])  # Original test row
print(f"\nTrue Label: {true_label}")
print(f"Predicted Label: {predicted_label[0]}")


First Test Row (original features):
loan_amnt                      15000.00
funded_amnt                    15000.00
funded_amnt_inv                15000.00
term                               1.00
int_rate                          11.99
installment                      333.60
sub_grade                          5.00
annual_inc                     50000.00
verification_status                0.00
dti                               15.84
delinq_2yrs                        0.00
inq_last_6mths                     0.00
open_acc                          10.00
pub_rec                            0.00
revol_bal                      17637.00
revol_util                        53.60
total_acc                         17.00
initial_list_status                1.00
out_prncp                      14816.28
out_prncp_inv                  14816.28
total_pymnt                      328.60
total_pymnt_inv                  328.60
total_rec_prncp                  183.72
total_rec_int                    144.88
tota

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [5]:
pip install matplotlib pyngrok joblib

Note: you may need to restart the kernel to use updated packages.


# **Web server Flask Application**

In [ ]:
from flask import Flask, request, render_template_string, send_file
import joblib
import numpy as np
import pandas as pd
from pyngrok import ngrok
import socket

app = Flask(__name__, static_folder='static')

# Load the pre-trained model, PCA, and scaler
model = joblib.load('loan_model.pkl')
pca = joblib.load('pca_model.pkl')
scaler = joblib.load('scaler.pkl')

# Set up ngrok with your auth token
ngrok.set_auth_token('2fsJOfECrU5047Dx6yFHRoGNKFf_2QJgtV1GhxZ64g8gRzX3')

# Load your dataset
df = pd.read_csv('/kaggle/input/terativeimputer-credit1/IterativeImputer_credit(1).csv')
X = df.drop(columns=['loan_status'])  # Assuming the dataset has a 'loan_status' column
y = df['loan_status']

# Feature names
feature_names = X.columns.tolist()

# Status code mapping
STATUS_CODE_NAME = {
    0: 'Charged Off',
    1: 'Current',
    2: 'Default',
    3: 'Does not meet the credit policy. Status: Charged Off',
    4: 'Does not meet the credit policy. Status: Fully Paid',
    5: 'Fully Paid',
    6: 'In Grace Period',
    7: 'Issued',
    8: 'Late (16-30 days)',
    9: 'Late (31-120 days)'
}

# Priority mapping for loan status
priority_dictionary = {
    'Fully Paid': 1,
    'Does not meet the credit policy. Status: Fully Paid': 2,
    'Current': 3,
    'Issued': 4,
    'In Grace Period': 5,
    'Late (16-30 days)': 6,
    'Late (31-120 days)': 7,
    'Default': 8,
    'Charged Off': 9,
    'Does not meet the credit policy. Status: Charged Off': 10
}

@app.route("/background_image")
def background_image():
    image_path = "/kaggle/input/istockphoto/istockphoto-1145371340-612x612.jpg"
    return send_file(image_path, mimetype='image/webp')

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        return predict()

    # Fetch row index 4 values
    row_index = 4
    row_4 = X.iloc[row_index]

    num_features_per_row = 4
    grouped_form_fields = [
        '<div class="row">' + ''.join(
            f'<div class="column"><label for="{feature}">{feature}:</label>'
            f'<input type="number" name="{feature}" value="{row_4[feature]}" required></div>'
            for feature in feature_names[i:i + num_features_per_row]
        ) + '</div>'
        for i in range(0, len(feature_names), num_features_per_row)
    ]

    return render_template_string(f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Loan Prediction</title>
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
        <style>
            body {{
                font-family: 'Arial', sans-serif;
                background-image: url('{{{{ url_for('background_image') }}}}');
                background-size: cover;
                background-attachment: fixed;
                text-align: center;
                padding: 50px;
                margin: 0;
                color: #333;
            }}
            h1 {{
                color: #ffffff;
                text-shadow: 2px 2px 8px rgba(0, 0, 0, 0.7);
                animation: fadeIn 1s ease-in-out;
                font-size: 2.5rem;
            }}
            .row {{
                display: flex;
                flex-wrap: wrap;
                justify-content: center;
                margin: 10px 0;
            }}
            .column {{
                flex: 1 1 200px;
                margin: 5px;
            }}
            form {{
                display: inline-block;
                padding: 20px;
                background-color: rgba(255, 255, 255, 0.9);
                border-radius: 10px;
                box-shadow: 0 4px 20px rgba(0, 0, 0, 0.2);
                max-width: 600px;
                width: 100%;
                transition: transform 0.3s;
            }}
            form:hover {{
                transform: scale(1.02);
            }}
            input, label {{
                width: 100%;
            }}
            input[type="number"] {{
                margin-top: 5px;
                padding: 10px;
                border: 1px solid #ccc;
                border-radius: 5px;
                transition: border-color 0.3s;
                font-size: 1rem;
            }}
            input[type="number"]:focus {{
                border-color: #28a745;
                outline: none;
            }}
            input[type="submit"] {{
                background-color: #28a745;
                color: white;
                cursor: pointer;
                padding: 10px 20px;
                margin-top: 20px;
                border: none;
                border-radius: 5px;
                font-size: 1.2rem;
                transition: background-color 0.3s, transform 0.2s;
            }}
            input[type="submit"]:hover {{
                background-color: #218838;
                transform: scale(1.05);
            }}
        </style>
    </head>
    <body>
        <h1>Loan Prediction System</h1>
        <form method="POST">
            {''.join(grouped_form_fields)}
            <input type="submit" value="Predict Loan Status">
        </form>
    </body>
    </html>
    ''')

def predict():
    try:
        # Retrieve the input features from the form
        features = [float(request.form[feature]) for feature in feature_names]
        final_features = np.array(features).reshape(1, -1)

        # Perform scaling and PCA
        scaled_features = scaler.transform(final_features)
        final_features_pca = pca.transform(scaled_features)

        # Make prediction
        prediction = model.predict(final_features_pca)[0]
        loan_status = STATUS_CODE_NAME.get(prediction, "Unknown Status")
        priority = priority_dictionary.get(loan_status, "Unknown Priority")

    except Exception as e:
        loan_status = f"Error: {str(e)}"
        priority = "Unknown"

    return render_template_string(f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Loan Prediction Result</title>
        <style>
            body {{
                font-family: 'Arial', sans-serif;
                background-image: url('{{{{ url_for('background_image') }}}}');
                background-size: cover;
                background-attachment: fixed;
                display: flex;
                justify-content: center;
                align-items: center;
                min-height: 100vh;
                margin: 0;
            }}
            .result-container {{
                background-color: #ffffff;
                border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                padding: 30px;
                max-width: 500px;
                width: 100%;
                text-align: center;
            }}
            h1 {{
                color: #333;
                font-size: 2rem;
                margin-bottom: 20px;
            }}
            p {{
                font-size: 1.2rem;
                margin-bottom: 10px;
                color: #666;
            }}
            button {{
                background-color: #28a745;
                color: #fff;
                padding: 12px 24px;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                font-size: 1rem;
                transition: background-color 0.3s;
            }}
            button:hover {{
                background-color: #218838;
            }}
            .back-button {{
                margin-top: 20px;
            }}
        </style>
    </head>
    <body>
        <div class="result-container">
            <h1>Loan Prediction Result</h1>
            <p><strong>Status:</strong> {loan_status}</p>
            <p><strong>Priority Level:</strong> {priority}</p>
            <br>
            <a href="{{{{ url_for('index') }}}}"><button class="back-button">Back to Prediction</button></a>
        </div>
    </body>
    </html>
    ''')

def get_open_port():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(("", 0))
    port = sock.getsockname()[1]
    sock.close()
    return port

if __name__ == "__main__":
    port = get_open_port()
    ngrok_tunnel = ngrok.connect(port)
    print('NGROK Tunnel URL:', ngrok_tunnel.public_url)
    app.run(port=port)


NGROK Tunnel URL: https://f6aa-34-90-19-193.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


# **import libraries**

In [7]:
from flask import Flask, request, render_template_string, send_file
import joblib
import numpy as np
import pandas as pd
from pyngrok import ngrok
import socket

# **Flask Application Deployment Model**

In [8]:
app = Flask(__name__, static_folder='static')
# Set up ngrok with your auth token
ngrok.set_auth_token('2fsJOfECrU5047Dx6yFHRoGNKFf_2QJgtV1GhxZ64g8gRzX3')


# **load models which We created**

In [9]:
# Load the pre-trained model, PCA, and scaler
model = joblib.load('loan_model.pkl')
pca = joblib.load('pca_model.pkl')
scaler = joblib.load('scaler.pkl')

# **Data_set_credit_Risk**

In [10]:
# Load your dataset
df = pd.read_csv('/kaggle/input/terativeimputer-credit1/IterativeImputer_credit(1).csv')
X = df.drop(columns=['loan_status'])  # Assuming the dataset has a 'loan_status' column
y = df['loan_status']

# Feature names
feature_names = X.columns.tolist()

# Status code mapping
STATUS_CODE_NAME = {
    0: 'Charged Off',
    1: 'Current',
    2: 'Default',
    3: 'Does not meet the credit policy. Status: Charged Off',
    4: 'Does not meet the credit policy. Status: Fully Paid',
    5: 'Fully Paid',
    6: 'In Grace Period',
    7: 'Issued',
    8: 'Late (16-30 days)',
    9: 'Late (31-120 days)'
}

# Priority mapping for loan status
priority_dictionary = {
    'Fully Paid': 1,
    'Does not meet the credit policy. Status: Fully Paid': 2,
    'Current': 3,
    'Issued': 4,
    'In Grace Period': 5,
    'Late (16-30 days)': 6,
    'Late (31-120 days)': 7,
    'Default': 8,
    'Charged Off': 9,
    'Does not meet the credit policy. Status: Charged Off': 10
}

# **page1: application Form for inputs_features**

In [11]:
@app.route("/background_image")
def background_image():
    image_path = "/kaggle/input/istockphoto/istockphoto-1145371340-612x612.jpg"
    return send_file(image_path, mimetype='image/webp')

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        return predict()

    # Fetch row index 4 values
    row_index = 4
    row_4 = X.iloc[row_index]

    num_features_per_row = 4
    grouped_form_fields = [
        '<div class="row">' + ''.join(
            f'<div class="column"><label for="{feature}">{feature}:</label>'
            f'<input type="number" name="{feature}" value="{row_4[feature]}" required></div>'
            for feature in feature_names[i:i + num_features_per_row]
        ) + '</div>'
        for i in range(0, len(feature_names), num_features_per_row)
    ]

    return render_template_string(f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Loan Prediction</title>
        <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
        <style>
            body {{
                font-family: 'Arial', sans-serif;
                background-image: url('{{{{ url_for('background_image') }}}}');
                background-size: cover;
                background-attachment: fixed;
                text-align: center;
                padding: 50px;
                margin: 0;
                color: #333;
            }}
            h1 {{
                color: #ffffff;
                text-shadow: 2px 2px 8px rgba(0, 0, 0, 0.7);
                animation: fadeIn 1s ease-in-out;
                font-size: 2.5rem;
            }}
            .row {{
                display: flex;
                flex-wrap: wrap;
                justify-content: center;
                margin: 10px 0;
            }}
            .column {{
                flex: 1 1 200px;
                margin: 5px;
            }}
            form {{
                display: inline-block;
                padding: 20px;
                background-color: rgba(255, 255, 255, 0.9);
                border-radius: 10px;
                box-shadow: 0 4px 20px rgba(0, 0, 0, 0.2);
                max-width: 600px;
                width: 100%;
                transition: transform 0.3s;
            }}
            form:hover {{
                transform: scale(1.02);
            }}
            input, label {{
                width: 100%;
            }}
            input[type="number"] {{
                margin-top: 5px;
                padding: 10px;
                border: 1px solid #ccc;
                border-radius: 5px;
                transition: border-color 0.3s;
                font-size: 1rem;
            }}
            input[type="number"]:focus {{
                border-color: #28a745;
                outline: none;
            }}
            input[type="submit"] {{
                background-color: #28a745;
                color: white;
                cursor: pointer;
                padding: 10px 20px;
                margin-top: 20px;
                border: none;
                border-radius: 5px;
                font-size: 1.2rem;
                transition: background-color 0.3s, transform 0.2s;
            }}
            input[type="submit"]:hover {{
                background-color: #218838;
                transform: scale(1.05);
            }}
        </style>
    </head>
    <body>
        <h1>Loan Prediction System</h1>
        <form method="POST">
            {''.join(grouped_form_fields)}
            <input type="submit" value="Predict Loan Status">
        </form>
    </body>
    </html>
    ''')

# **page2: loan prediction**

In [12]:
def predict():
    try:
        # Retrieve the input features from the form
        features = [float(request.form[feature]) for feature in feature_names]
        final_features = np.array(features).reshape(1, -1)

        # Perform scaling and PCA
        scaled_features = scaler.transform(final_features)
        final_features_pca = pca.transform(scaled_features)

        # Make prediction
        prediction = model.predict(final_features_pca)[0]
        loan_status = STATUS_CODE_NAME.get(prediction, "Unknown Status")
        priority = priority_dictionary.get(loan_status, "Unknown Priority")

    except Exception as e:
        loan_status = f"Error: {str(e)}"
        priority = "Unknown"

    return render_template_string(f'''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Loan Prediction Result</title>
        <style>
            body {{
                font-family: 'Arial', sans-serif;
                background-image: url('{{{{ url_for('background_image') }}}}');
                background-size: cover;
                background-attachment: fixed;
                display: flex;
                justify-content: center;
                align-items: center;
                min-height: 100vh;
                margin: 0;
            }}
            .result-container {{
                background-color: #ffffff;
                border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                padding: 30px;
                max-width: 500px;
                width: 100%;
                text-align: center;
            }}
            h1 {{
                color: #333;
                font-size: 2rem;
                margin-bottom: 20px;
            }}
            p {{
                font-size: 1.2rem;
                margin-bottom: 10px;
                color: #666;
            }}
            button {{
                background-color: #28a745;
                color: #fff;
                padding: 12px 24px;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                font-size: 1rem;
                transition: background-color 0.3s;
            }}
            button:hover {{
                background-color: #218838;
            }}
            .back-button {{
                margin-top: 20px;
            }}
        </style>
    </head>
    <body>
        <div class="result-container">
            <h1>Loan Prediction Result</h1>
            <p><strong>Status:</strong> {loan_status}</p>
            <p><strong>Priority Level:</strong> {priority}</p>
            <br>
            <a href="{{{{ url_for('index') }}}}"><button class="back-button">Back to Prediction</button></a>
        </div>
    </body>
    </html>
    ''')


# **web server Link**

In [ ]:
def get_open_port():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(("", 0))
    port = sock.getsockname()[1]
    sock.close()
    return port

if __name__ == "__main__":
    port = get_open_port()
    ngrok_tunnel = ngrok.connect(port)
    print('NGROK Tunnel URL:', ngrok_tunnel.public_url)
    app.run(port=port)